In [104]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn

# Read employees dataset
df_employee = pd.read_csv('data/employees4.csv', na_values=['\\N'], on_bad_lines='skip', infer_datetime_format=True, parse_dates=['Date_Joined', 'Date_Resigned', 'Inactive_Date'])
df_employee.info()
print(f'Total employees {str(df_employee["Employee_Code"].nunique())}')
print(f'Earliest join date {str(df_employee["Date_Joined"].min())}')
print(f'Latest join date {str(df_employee["Date_Joined"].max())}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Employee_No          1261 non-null   int64         
 1   Employee_Code        1261 non-null   int64         
 2   Name                 1261 non-null   object        
 3   Title                1261 non-null   object        
 4   Address_Line_1       1261 non-null   object        
 5   Address_Line_2       1181 non-null   object        
 6   Address_Line_3       952 non-null    object        
 7   Date_of_Birth        1261 non-null   object        
 8   Gender               1261 non-null   object        
 9   Religion_ID          1261 non-null   int64         
 10  Marital_Status       1170 non-null   object        
 11  Designation_ID       1261 non-null   int64         
 12  Date_Joined          1261 non-null   datetime64[ns]
 13  Date_Resigned        955 non-null

In [105]:
# Read leaves dataset
df_leaves = pd.read_csv('data/leaves_clean.csv', names=["Employee_Code", "Leave_Date", "Type", "Applied_Date", "Lieu_Leave_Date", "Remarks", "Leave_Category"])
df_leaves_summary = df_leaves.groupby('Employee_Code')['Leave_Date'].count()
df_leaves_summary['Employee_Code'] = df_leaves['Employee_Code'].astype('object')
df_leaves_summary.head()
df_leaves_summary.info()

<class 'pandas.core.series.Series'>
Index: 1743 entries, 1001 to Employee_Code
Series name: Leave_Date
Non-Null Count  Dtype 
--------------  ----- 
1743 non-null   object
dtypes: object(1)
memory usage: 27.2+ KB


In [106]:
# Group two datasets

df_joined = pd.merge(df_employee, df_leaves_summary, on='Employee_Code'  , how='inner')
df_joined

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
temp_plot = df_employee['Employment_Type'].value_counts().plot(kind='bar')
temp_plot.bar_label(temp_plot.containers[0], labels=df_employee['Employment_Type'].value_counts())
plt.show()

plt.pie(df_employee['Status'].value_counts(), labels=df_employee['Status'].unique(), autopct='%1.1f%%')
plt.legend()
plt.show()

seaborn.pairplot(df_employee)
#temp_plot = df['Designation'].value_counts().plot(kind='pie')
#plt.legend()
#plt.show()

In [ ]:
df_employee_inactive_contract_based = df_employee.loc[(df_employee['Status'] == 'Inactive') & (df_employee['Employment_Type'] == 'Contarct Basis')]
df_employee_inactive_contract_based['Date_Joined'] = pd.to_datetime(df_employee_inactive_contract_based['Date_Joined'])
df_employee_inactive_contract_based['Date_Resigned'] = pd.to_datetime(df_employee_inactive_contract_based['Date_Resigned'], errors='coerce')
df_employee_inactive_contract_based['Employment_Duration'] = df_employee_inactive_contract_based['Date_Resigned'] - df_employee_inactive_contract_based['Date_Joined']
df_employee_inactive_contract_based['Employment_Duration'].describe()

In [ ]:
df_employee_inactive_permanent = df_employee.loc[(df_employee['Status'] == 'Inactive') & (df_employee['Employment_Type'] == 'Permanant')]
df_employee_inactive_permanent['Date_Joined'] = pd.to_datetime(df_employee_inactive_permanent['Date_Joined'])
df_employee_inactive_permanent['Date_Resigned'] = pd.to_datetime(df_employee_inactive_permanent['Date_Resigned'], errors='coerce')
df_employee_inactive_permanent['Employment_Duration'] = df_employee_inactive_permanent['Date_Resigned'] - df_employee_inactive_permanent['Date_Joined']
df_employee_inactive_permanent['Employment_Duration'].describe()

In [ ]:
df_employee_inactive = df_employee.loc[(df_employee['Status'] == 'Inactive')]
df_employee_inactive['Date_Joined'] = pd.to_datetime(df_employee_inactive['Date_Joined'])
df_employee_inactive['Date_Resigned'] = pd.to_datetime(df_employee_inactive['Date_Resigned'], errors='coerce')
df_employee_inactive['Employment_Duration'] = df_employee_inactive['Date_Resigned'] - df_employee_inactive['Date_Joined']
df_employee_inactive['Employment_Duration'].describe()

In [ ]:
from datetime import datetime
df__employee_active_contract_basis = df_employee.loc[(df_employee['Status'] == 'Active') & (df_employee['Employment_Type'] == 'Contarct Basis')]
df__employee_active_contract_basis['Date_Joined'] = pd.to_datetime(df__employee_active_contract_basis['Date_Joined'])
today_timestamp = pd.Timestamp(datetime.today().date())
df__employee_active_contract_basis['Employment_Duration'] = today_timestamp - df__employee_active_contract_basis['Date_Joined']
df__employee_active_contract_basis['Employment_Duration'].describe()